In [1]:
# ------------------------------------------------------------------------------------------------
# Paquetes.
# ------------------------------------------------------------------------------------------------

import pandas as pd
import numpy as np
import math 
import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter        
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import NamedStyle, Font, PatternFill, Border, Side, Alignment, Color
from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.worksheet.worksheet import Worksheet
import pyautogui                     
import time
import pygetwindow as gw


In [2]:

import sys
sys.path.append('c:/Users/tomas/Documents/Programación/Proyectos/Paquetes')
import Forpy as fg
import Excpy as ex
import Frampy as fr
import Stringpy as st


In [3]:

# ------------------------------------------------------------------------------------------------
# Variables.
# ------------------------------------------------------------------------------------------------

# Archivo Proveedores.
Proveedores = 'J:/My Drive/Forraje/Proveedores.xlsx'
df_Proveedores = pd.read_excel(Proveedores, sheet_name='Centro')

# Proveedores.
Lista_Proveedores = ['Benavídez', 'Gallo', 'Anea', 'Pilar', 'Tortuguitas', 'Del Viso',
                     'Chica Glade', 'San Martín', 'Hipocampus', 'Mundo Mascotas', 'Yaguar',
                     'King Clor', 'Diserquim', 'Moreno', 'Roberto', 'Viejo Gets', 'Viejo Cuero', 'Mis Nietos', 
                     'Profesional Vet', 'Premier', 'Dacam']

# Archivo Nex.
Nex = 'J:/My Drive/Forraje/Exportar.xls'
df_Nex = pd.read_excel(Nex)

# Archivo Granel.
Granel = 'J:/My Drive/Forraje/Granel.xlsx'
df_Granel = pd.read_excel(Granel)

# Archivo Fraccionados.
Fraccion = 'J:/My Drive/Forraje/Fraccionables.xls'
df_Fraccion = pd.read_excel(Fraccion)

# Ofertas y precios establecidos.
Ofertas = {}


In [4]:
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# PROCESAMIENTO DEL ARCHIVO PROVEEDORES.XLSX
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------

# ------------------------------------------------------------------------------------------------
# Formatear el texto de las columnas de proveedores.
# ------------------------------------------------------------------------------------------------

df_Proveedores = fg.Column_Provider_Processing(df_Proveedores, Lista_Proveedores)


In [5]:


# ------------------------------------------------------------------------------------------------
# Hallar precio mínimo y proveedor.
# ------------------------------------------------------------------------------------------------

Minimo_Sin_Cero = df_Proveedores[Lista_Proveedores].replace(0, np.nan).min(axis=1, skipna=True)
df_Proveedores['Precio'] = Minimo_Sin_Cero

df_Proveedores = fg.Find_Best_Provider(df_Proveedores, Lista_Proveedores)



C:\Users\tomas\Documents\Programación\Github\Patricionog\Modulio\Forpy.py:104: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Benavídez' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[Index, "Proveedor"] = Provider


In [6]:

# ------------------------------------------------------------------------------------------------
# Agregar columnas con los precios y proveedores más baratos.
# ------------------------------------------------------------------------------------------------

Columns = ['Codigo', 'Descripcion', 'Precio', 'Categoria', 'Proveedor']

df_Proveedores = df_Proveedores[Columns].copy()
df_Proveedores.rename(columns={'Precio': 'Costo'}, inplace=True)



In [7]:

# ------------------------------------------------------------------------------------------------
# Agregar columnas con Markups y tipo de Unidad.
# ------------------------------------------------------------------------------------------------

df_Proveedores['Markup %'] = df_Proveedores['Categoria'].map(lambda x: fg.Markup_And_Unity[x]['Markup %'])
df_Proveedores['Unidad'] = df_Proveedores['Categoria'].map(lambda x: fg.Markup_And_Unity[x]['Unidad'])



In [8]:

# ------------------------------------------------------------------------------------------------
# Filtrar df.
# ------------------------------------------------------------------------------------------------

Columns = ['Descripcion','Costo','Markup %','Proveedor','Categoria', 'Unidad']
df_Proveedores = df_Proveedores[Columns]



In [9]:


# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# PROCESAMIENTO DEL ARCHIVO EXPORTAR.XLS, CATÁLOGO DEL SISTEMA NEXTAR
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------

# ------------------------------------------------------------------------------------------------
# Renombrar columnas y filtrar df.
# ------------------------------------------------------------------------------------------------

df_Nex.rename(columns={'Costo Unitário': 'Costo'}, inplace=True)

df_Nex = fr.Add_Word_To_Name_Columns(df_Nex, Word = "Nex", Separator = '_')


In [10]:

# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# DATAFRAME FINAL.
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------

# ------------------------------------------------------------------------------------------------
# Verificar que los dos df tienen el mismo largo y componentes.
# ------------------------------------------------------------------------------------------------

Is_Same_Lenght = fr.Compare_Columns(df_Nex, 'Descripcion_Nex', df_Proveedores, 'Descripcion')

if len(Is_Same_Lenght) != 0:
    print(Is_Same_Lenght)
    raise Exception("Los dfs no tienen la misma longitud o tienen diferencias en los nombres de los productos.")

Columns = ['Codigo_Nex','Costo_Nex','Precio_Nex']
df_Nex = df_Nex[Columns]

df = pd.concat([df_Nex, df_Proveedores], axis=1)


df.rename(columns={'Codigo_Nex': 'Codigo'}, inplace=True)
df.rename(columns={'Costo': 'Costo_Prov'}, inplace=True)
df.rename(columns={'Precio': 'Precio_Nex'}, inplace=True)

Columns_New_Order = ["Codigo", "Descripcion", "Costo Final", "Precio_Nex", "Costo_Nex", "Costo_Prov", "Markup %", "Categoria", "Proveedor"]

df = df.reindex(columns=Columns_New_Order)


In [11]:

# ------------------------------------------------------------------------------------------------
# Calcular variables nuevas.
# ------------------------------------------------------------------------------------------------

df['Costo Final'] = df.apply(lambda row: max(row['Costo_Nex'], row['Costo_Prov']), axis=1).fillna(0)



In [12]:

# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# CALCULAR PRECIOS A GRANEL.
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------

df_Granel = fr.Get_Last_Number_Of_String_Column(df_Granel, 'Bolsa', 'Kilos')

df_Granel = fr.Match_And_Copy_Column_Values(df_Granel, df, 'Bolsa', 'Costo Bolsa', 'Descripcion',  'Costo Final')

df_Granel = df_Granel.replace('', 0)

df_Granel = fr.Convert_Type_Of_Columns(df_Granel, ['Kilos', 'Costo Bolsa', 'Costo Final'], "float")

df_Granel['Costo Kilo'] = df_Granel['Costo Bolsa']/df_Granel['Kilos']

df = fr.Match_And_Copy_Column_Values(df, df_Granel, 'Descripcion', 'Costo_Prov', 'Granel', 'Costo Kilo')



In [13]:


# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# CALCULAR PRECIOS DE FRACCIONADOS.
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------

df_Fraccion = fr.Get_Last_Number_Of_String_Column(df_Fraccion, 'Fraccionable', 'Unidades')

df_Fraccion = fr.Match_And_Copy_Column_Values(df_Fraccion, df, 'Fraccionable', 'Costo Paquete', 'Descripcion', 'Costo Final')

df_Fraccion = df_Fraccion.replace('', 0)

df_Fraccion = fr.Convert_Type_Of_Columns(df_Fraccion, ['Unidades', 'Costo Paquete'], "float")

df_Fraccion['Costo Unidad'] = df_Fraccion['Costo Paquete']/df_Fraccion['Unidades']

In [14]:
# ------------------------------------------------------------------------------------------------
# Casos especiales: Ivomec.
# ------------------------------------------------------------------------------------------------

df_Fraccion.loc[df_Fraccion['Fraccionable'] == 'Ivomec x 500 cc.', 'Costo Unidad'] += df_Fraccion.loc[df_Fraccion['Fraccionable'] == 'Jeringa x 100 u.', 'Costo Unidad'].values[0]


In [15]:
# ------------------------------------------------------------------------------------------------
# Casos especiales: A otra parte.
# ------------------------------------------------------------------------------------------------

# Encontrar la fila donde la columna "Descripcion" es igual a "Envase x 250 cc."
Fila_Botellita = df.loc[df['Descripcion'] == 'Envase x 250 cc.']

# Extraer el valor de la columna "Costo Final" de la fila encontrada
Costo_Botellita = Fila_Botellita['Costo Final'].values[0]

# Utiliza el método .iloc para acceder al primer elemento de la columna y asignarle el nuevo valor
df_Fraccion.loc[df_Fraccion['Fraccionable'] == 'A Otra Parte Concentrado x 600 cc.', 'Costo Unidad'] = df_Fraccion['Costo Unidad'].iloc[0]*25 + Costo_Botellita

In [16]:
# ------------------------------------------------------------------------------------------------
# Casos especiales: Bozales.
# ------------------------------------------------------------------------------------------------

Costo = df_Fraccion.loc[df_Fraccion['Fraccionable'] == 'Bozal Canasta Pack x 7 u.', 'Costo Paquete'].values[0]

df_Fraccion.loc[df_Fraccion['Unidad'] == 'Bozal Canasta N° 1', 'Costo Unidad'] = Costo * 0.10
df_Fraccion.loc[df_Fraccion['Unidad'] == 'Bozal Canasta N° 2', 'Costo Unidad'] = Costo * 0.12
df_Fraccion.loc[df_Fraccion['Unidad'] == 'Bozal Canasta N° 3', 'Costo Unidad'] = Costo * 0.14
df_Fraccion.loc[df_Fraccion['Unidad'] == 'Bozal Canasta N° 4', 'Costo Unidad'] = Costo * 0.16
df_Fraccion.loc[df_Fraccion['Unidad'] == 'Bozal Canasta N° 5', 'Costo Unidad'] = Costo * 0.20
df_Fraccion.loc[df_Fraccion['Unidad'] == 'Bozal Canasta N° 6', 'Costo Unidad'] = Costo * 0.22

In [17]:
# Adicionamos los costos por unidad al df.
df = fr.Match_And_Copy_Column_Values(df, df_Fraccion, 'Descripcion', 'Costo_Prov', 'Unidad', 'Costo Unidad')

In [18]:
# Convertir Markup a float.
df['Markup %'] = df['Markup %'].astype(float)

In [19]:
# ------------------------------------------------------------------------------------------------
# Casos especiales: Pulmax Perro.
# ------------------------------------------------------------------------------------------------

Pulmax = ['Pipeta Pulmax 02-05 kg.', 'Pipeta Pulmax 05-10 kg.', 'Pipeta Pulmax 10-20 kg.', 
          'Pipeta Pulmax 20-40 kg.', 'Pipeta Pulmax 40-60 kg.']
    
Contador = 50
for Pipeta in Pulmax:
    df.loc[df['Descripcion'] == Pipeta, 'Markup %'] = Contador
    Contador = Contador + 5

In [20]:
# ------------------------------------------------------------------------------------------------
# Casos especiales: Pulmax Gato.
# ------------------------------------------------------------------------------------------------

Pulmax_Gato = ['Pipeta Pulmax Gato 00-04 kg.', 'Pipeta Pulmax Gato 04-08 kg.']

Contador = 50 
for Pipeta in Pulmax_Gato:
    df.loc[df['Descripcion'] == Pipeta, 'Markup %'] = Contador
    Contador = Contador + 5

In [21]:
# ------------------------------------------------------------------------------------------------
# Casos especiales: Hectopar Perro.
# ------------------------------------------------------------------------------------------------

Hectopar = ['Pipeta Hectopar 00-04 kg.', 'Pipeta Hectopar 05-10 kg.', 'Pipeta Hectopar 10-25 kg.',
            'Pipeta Hectopar 25-40 kg.', 'Pipeta Hectopar 40-60 kg.']

Contador = 50 
for Pipeta in Hectopar:
    df.loc[df['Descripcion'] == Pipeta, 'Markup %'] = Contador
    Contador = Contador + 5

In [22]:
# ------------------------------------------------------------------------------------------------
# Casos especiales: Hectopar Gato.
# ------------------------------------------------------------------------------------------------

# Hectopar Gato.
Hectopar_Gato = ['Pipeta Hectopar Gato 00-04 kg.', 'Pipeta Hectopar Gato 05-08 kg.']

Contador = 50 
for Pipeta in Hectopar_Gato:
    df.loc[df['Descripcion'] == Pipeta, 'Markup %'] = Contador
    Contador = Contador + 5

In [23]:
# Columna Precio Proveedores.

df['Precio_Prov'] = df['Costo_Prov']* (1 + (df['Markup %']/100))

In [24]:
# Llenamos df de 0 si hay NaN.
df = df.fillna(0)

In [25]:
# Columna de Relaciones.

df['Precio_Dif'] = df['Precio_Prov'] / df['Precio_Nex']
df['Costo_Dif'] = df['Costo_Prov'] / df['Costo_Nex']

In [26]:
# Columna Costo Final.
df['Costo_Fin'] = df.apply(lambda row: max(row['Costo_Nex'], row['Costo_Prov']), axis=1)

In [27]:
# Columna Precio Final.
df['Precio_Fin'] = df.apply(lambda row: max(row['Precio_Nex'], row['Precio_Prov']), axis=1)

In [28]:
# Redondeo de Precio Final.
df['Precio_Fin'] = df['Precio_Fin'].apply(lambda x: math.ceil(x / 50) * 50)

In [29]:
# Columna Markup Final.
df['Markup_Fin'] = (df['Precio_Fin']/df['Costo_Fin']) - 1

In [30]:
# Columna Markup.
df['Markup %'] = df['Markup %']/100

In [31]:
# Lista de productos con ofertas.
Productos_Ofertas = list(Ofertas.keys())

In [ ]:
# Arreglamos precios de ofertas.
for Producto in Productos_Ofertas:
    df.loc[df['Descripcion'] == Producto, 'Precio_Fin'] = Ofertas[Producto]

In [33]:
# Ordenar columnas.
Orden = ['Descripcion', 'Precio_Fin', 'Costo_Fin', 'Markup_Fin',
         'Costo_Nex', 'Costo_Prov', 'Precio_Nex', 'Precio_Prov', 
         'Precio_Dif','Costo_Dif', 
         'Markup %', 'Proveedor', 'Categoria']

In [34]:
# Reasigna el DataFrame con el nuevo orden de columnas
df = df[Orden]

In [35]:
# Creamos un nuevo DataFrame con los precios a aumentar.
df_Aumentos = df[Orden]

In [36]:
# Filtramos los productos en los que el Precio_Prov es mayor al Precio_Nex.
df_Aumentos = df_Aumentos[df_Aumentos['Precio_Prov'] > df_Aumentos['Precio_Nex']]

In [37]:
# Reiniciar index.
df_Aumentos.reset_index(drop=True, inplace=True)

In [38]:
# Cantidad de productos a aumentar.
Cantidad_Productos_A_Aumentar = len(df_Aumentos)

In [39]:
# Aumentos.
#fg.Aumentar_Nex(df_Aumentos, Cantidad_Productos_A_Aumentar)

In [43]:
# Volvemos a cargar el archivo de proveedores.
Proveedores = 'J:/My Drive/Forraje/Proveedores.xlsx'

# Crear el DataFrame utilizando esas columnas
df_Proveedores = pd.read_excel(Proveedores, sheet_name='Centro')

In [44]:
# Lista de proveedores.
Lista_Proveedores = ['Benavídez', 'Gallo', 'Anea', 'Pilar', 'Tortuguitas', 'Del Viso',
                     'Chica Glade', 'San Martín', 'Hipocampus', 'Mundo Mascotas', 'Yaguar',
                     'King Clor', 'Diserquim', 'Moreno', 'Roberto', 'Viejo Gets', 'Viejo Cuero', 'Mis Nietos', 
                     'Profesional Vet', 'Premier', 'Dacam']

In [45]:
# Encuentra el mínimo excluyendo los valores de cero.
Primer_Minimo = df_Proveedores[Lista_Proveedores].replace(0, np.nan).min(axis=1, skipna=True)

In [46]:
# Encuentra el segundo mínimo excluyendo los valores de cero
Segundo_Minimo = df_Proveedores[Lista_Proveedores].apply(lambda row: sorted(filter(lambda x: x != 0, row))[:2][1] if len([x for x in row if x != 0]) >= 2 else 0, axis=1)

In [47]:
# Encuentra el segundo mínimo excluyendo los valores de cero
Tercer_Minimo = df_Proveedores[Lista_Proveedores].apply(lambda row: sorted(filter(lambda x: x != 0, row))[:3][2] if len([x for x in row if x != 0]) >= 3 else 0, axis=1)


In [48]:
# Actualiza la columna 'Precios 1, 2 y 3' con los mínimos calculados.
df_Proveedores['Precio 1'] = Primer_Minimo
df_Proveedores['Precio 2'] = Segundo_Minimo
df_Proveedores['Precio 3'] = Tercer_Minimo

In [49]:
# Columnas de diferencias.
df_Proveedores['Diferencia (2°)'] = df_Proveedores['Precio 2'] - df_Proveedores['Precio 1']
df_Proveedores['Diferencia (3°)'] = df_Proveedores['Precio 3'] - df_Proveedores['Precio 1']


In [50]:
# Reemplazar NaN con 0
df_Proveedores['Precio 1'] = df_Proveedores['Precio 1'].fillna(0)
df_Proveedores['Precio 2'] = df_Proveedores['Precio 2'].fillna(0)
df_Proveedores['Precio 3'] = df_Proveedores['Precio 3'].fillna(0)
df_Proveedores['Diferencia (2°)'] = df_Proveedores['Diferencia (2°)'].fillna(0)
df_Proveedores['Diferencia (3°)'] = df_Proveedores['Diferencia (3°)'].fillna(0)

In [51]:
# Reemplazar valores negativos por 0.
df_Proveedores.loc[df_Proveedores['Diferencia (2°)'] < 0, 'Diferencia (2°)'] = '-'
df_Proveedores.loc[df_Proveedores['Diferencia (3°)'] < 0, 'Diferencia (3°)'] = '-'

C:\Users\tomas\AppData\Local\Temp\ipykernel_26744\3197066779.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_Proveedores.loc[df_Proveedores['Diferencia (2°)'] < 0, 'Diferencia (2°)'] = '-'
C:\Users\tomas\AppData\Local\Temp\ipykernel_26744\3197066779.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_Proveedores.loc[df_Proveedores['Diferencia (3°)'] < 0, 'Diferencia (3°)'] = '-'


In [52]:
# Iteramos sobre cada fila del DataFrame.
for Indice, Fila in df_Proveedores.iterrows():
    # Iteramos sobre cada columna de Lista_Proveedores.
    for Columna in Lista_Proveedores:
        # Comparamos el valor de "Precio" con el valor en la columna actual.
        if Fila["Precio 1"] == 0:
            df_Proveedores.at[Indice, "Proveedor 1"] = "-"
        elif Fila["Precio 1"] == Fila[Columna]:
            # Si coinciden, actualizamos la columna "Proveedores" con el nombre de la columna actual.
            df_Proveedores.at[Indice, "Proveedor 1"] = Columna
            # Salimos del bucle interno.
            break


In [53]:
# Iteramos sobre cada fila del DataFrame.
for Indice, Fila in df_Proveedores.iterrows():
    # Iteramos sobre cada Columna de Lista_Proveedores.
    for Columna in Lista_Proveedores:
        # Comparamos el valor de "Precio" con el valor en la columna actual.
        if Fila["Precio 2"] == 0:
            df_Proveedores.at[Indice, "Proveedor 2"] = "-"
        elif Fila["Precio 2"] == Fila[Columna]:
            # Si coinciden, actualizamos la Columna "Proveedores" con el nombre de la columna actual.
            df_Proveedores.at[Indice, "Proveedor 2"] = Columna
            # Salimos del bucle interno.
            break

In [54]:
# Iteramos sobre cada fila del DataFrame.
for Indice, Fila in df_Proveedores.iterrows():
    # Iteramos sobre cada Columna de Lista_Proveedores.
    for Columna in Lista_Proveedores:
        # Comparamos el valor de "Precio" con el valor en la columna actual.
        if Fila["Precio 3"] == 0:
            df_Proveedores.at[Indice, "Proveedor 3"] = "-"
        elif Fila["Precio 3"] == Fila[Columna]:
            # Si coinciden, actualizamos la columna "Proveedores" con el nombre de la columna actual.
            df_Proveedores.at[Indice, "Proveedor 3"] = Columna
            # Salimos del bucle interno
            break

In [55]:
df_Proveedores['Precio 1'] = df_Proveedores['Precio 1'].replace(0, '-')
df_Proveedores['Precio 2'] = df_Proveedores['Precio 2'].replace(0, '-')
df_Proveedores['Precio 3'] = df_Proveedores['Precio 3'].replace(0, '-')
df_Proveedores['Diferencia (2°)'] = df_Proveedores['Diferencia (2°)'].replace(0, '-')
df_Proveedores['Diferencia (3°)'] = df_Proveedores['Diferencia (3°)'].replace(0, '-')

In [56]:
# Se importa Database.

Database = 'J:/My Drive/Forraje/Marcas y subcategorías.xlsx'

# Crear el DataFrame utilizando esas columnas
df_Marcas = pd.read_excel(Database)

In [57]:
# Ordenar según la columna 'Descripcion' en orden ascendente.
df_Marcas = df_Marcas.sort_values(by='Descripcion')

In [58]:
# Resetear index del df.
df_Marcas = df_Marcas.reset_index(drop=True)

In [59]:
# Importar datos de marcas y subcategorias.

print(len(df_Marcas))
print(len(df_Proveedores))

if len(df_Marcas) == len(df_Proveedores):
    df_Proveedores['Marca'] = df_Marcas['Marca']
    df_Proveedores['Subcategoria'] = df_Marcas['Subcategoria']

2051
2051


In [60]:
# Nombres de las columnas en orden.
Orden = ['Codigo', 'Descripcion', 'Categoria', 'Subcategoria', 'Marca',
         'Proveedor 1', 'Precio 1', 
         'Proveedor 2', 'Precio 2', 'Diferencia (2°)',
         'Proveedor 3', 'Precio 3', 'Diferencia (3°)']

# Ordena el DataFrame
df_Proveedores = df_Proveedores[Orden]

In [61]:
# Guardar el DataFrame en un archivo.
Ruta_1 = f"J:/My Drive/Forraje/Baratos.xlsx"
df_Proveedores.to_excel(Ruta_1, index=False)